In [21]:
import torch
import os, glob
import random
import csv
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader


path = "../dataset/train.csv"
imgpath = "../dataset/images/"
# def read_csv(csvpath,imgpath):
#         # self.csv_path = csvpath
#         # self.imgpath = imgpath
#         train = pd.read_csv(csvpath)
#         train['image'] = train["Id"].map(lambda x:f"{imgpath}/{x:0>4}.jpg")
#         assert len(train["image"])==len(train["label"])
#         return list(train["image"]),list(train["label"])

# img,lael = read_csv(path,imgpath)
dir = os.listdir("../dataset/")
print(dir)

['images', 'sample_submission.csv', 'test.csv', 'train.csv']


In [49]:
class Mydataset(Dataset):
    def __init__(self,root,mode,resize):
        super(Mydataset,self).__init__()
        self.root = root
        self.resize = resize
        self.dirs = os.listdir(self.root)
        if mode == "train":
            self.imgpath,self.csv_path = self.root+self.dirs[0],self.root+self.dirs[-1]
            self.ImgPathList,self.LabelPathList = self.read_csv(self.csv_path,self.imgpath)
        if mode=="test":
            self.imgpath,self.csv_path = self.root+self.dirs[0],self.root+self.dirs[-2]
            self.ImgPathList,self.LabelPathList = self.read_csv(self.csv_path,self.imgpath)
        
    def read_csv(self,csvpath,imgpath):
        self.csv_path = csvpath
        self.imgpath = imgpath
        train = pd.read_csv(csvpath)
        train['image'] = train["Id"].map(lambda x:f"{imgpath}/{x:0>4}.jpg")
        assert len(train["image"])==len(train["label"])
        return list(train["image"]),list(train["label"])

    def __getitem__(self, index):
        img,label = self.ImgPathList[index],self.LabelPathList[index]
        img = Image.open(img).convert("RGB")
        transform = transforms.Compose([transforms.Resize((int(self.resize), int(self.resize))),
                                     transforms.RandomRotation(15),
                                     transforms.CenterCrop(self.resize),
                                     transforms.ToTensor(),  # 先变成tensor类型数据，然后在进行下面的标准化
                                     ])

        image = transform(img)
        label_dict = {"glass":0,"cup":1,"spoon":2,"plate":3,"knife":4,"fork":5}
        label = label_dict[label]
        label = torch.tensor(label) 
        return image, label

    def __len__(self):
        return len(self.LabelPathList)


def create_dataloader(data_path, mode, size, batch_size,
                    num_workers=0):  # 用一个函数加载上诉的数据，data_path、mode和size分别是以上定义的Dataset_self(）中的参数，batch_size是一次性输出多少张图像，num_worker是同时处理几张图像
    
    dataset = Mydataset(data_path, mode, size)
    dataloader = DataLoader(dataset, batch_size, num_workers)  # 使用pytorch中的dataloader函数得到数据

    return dataloader


if __name__=="__main__":
    data_path = "../dataset/"
    dataloader = create_dataloader(data_path,mode="train",size=224,batch_size=64)
    for img,label in dataloader:
        print(img.shape)
        print(label.shape)
        break

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [38]:
import torch
from torch import nn


# 先写好resnet的block块
class Res_block(nn.Module):
    def __init__(self, in_num, out_num, stride):
        super(Res_block, self).__init__()
        self.cov1 = nn.Conv2d(in_num, out_num, (3, 3), stride=stride,
                              padding=1)  # (3,3)  padding=1 则图像大小不变，stride为几图像就缩小几倍，能极大减少参数
        self.bn1 = nn.BatchNorm2d(out_num)
        self.cov2 = nn.Conv2d(out_num, out_num, (3, 3), padding=1)
        self.bn2 = nn.BatchNorm2d(out_num)

        self.extra = nn.Sequential(
            nn.Conv2d(in_num, out_num, (1, 1), stride=stride),
            nn.BatchNorm2d(out_num)
        )  # 使得输入前后的图像数据大小是一致的
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.bn1(self.cov1(x)))
        out = self.relu(self.bn2(self.cov2(out)))

        out = self.extra(x) + out
        return out


class Res_net(nn.Module):
    def __init__(self, num_class):
        super(Res_net, self).__init__()
        self.init = nn.Sequential(
            nn.Conv2d(3, 16, (3, 3)),
            nn.BatchNorm2d(16)
        )  # 预处理
        self.bn1 = Res_block(16, 32, 2)
        self.bn2 = Res_block(32, 64, 2)
        self.bn3 = Res_block(64, 128, 2)
        self.bn4 = Res_block(128, 256, 2)
        self.fl = nn.Flatten()
        self.linear1 = nn.Linear(100352, 10)
        self.linear2 = nn.Linear(10, num_class)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.init(x))
        # print('inint:',out.shape)
        out = self.bn1(out)
        # print('bn1:', out.shape)
        out = self.bn2(out)
        # print('bn2:', out.shape)
        out = self.bn3(out)
        # print('bn3:', out.shape)
        out = self.fl(out)
        #print('flatten:', out.shape)
        out = self.relu(self.linear1(out))
        # print('linear1:', out.shape)
        out = self.relu(self.linear2(out))
        # print('linear2:', out.shape)
        return out


# 测试
def main():
    x = torch.randn(1, 3, 224, 224)
    net = Res_net(6)
    out = net(x)
    print(out.shape)


if __name__ == '__main__':
    main()


torch.Size([1, 6])


In [54]:
from torch import optim
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate(model, loader):  # 计算每次训练后的准确率
    correct = 0
    model.cpu()
    total = len(loader.dataset)
    for x, y in loader:
        logits = model(x)
        pred = logits.argmax(dim=1)  # 得到logits中分类值（要么是[1,0]要么是[0,1]表示分成两个类别）
        correct += torch.eq(pred, y).sum().float().item()  # 用logits和标签label想比较得到分类正确的个数
    return correct / total


def train():
    epochs=10
    net = Res_net(6).to(device)
    data_path = "../dataset/"
    dataloader = create_dataloader(data_path,mode="train",size=224,batch_size=64)
    # test_dataloader = create_dataloader(data_path,"test",size=224,batch_size=64)
    optimizer = optim.Adam(net.parameters(),lr=0.00067)
    cirteron = nn.CrossEntropyLoss()
    losses=[]
    acc = []
    for epoch in range(epochs):
        net.train()
        pbar = tqdm(enumerate(dataloader),total=len(dataloader))
        epoch_loss = 0
        for i,(img,label) in pbar:
            out = net(img.to(device))
            out = out.cpu()
            loss = cirteron(out,label)
            losses.append(loss)

            optimizer.zero_grad()
            loss.backward()
            
            
            pre = torch.argmax(out)
            num_correct = (pre==label).sum()
            train_acc = float(num_correct)/float(img.shape[0])
            acc.append(train_acc)

            optimizer.step()

            pbar.set_description(f'Epoch [{epoch}/{epochs}]')
            pbar.set_postfix(loss=loss.item(),acc=train_acc)
        #     # if i%10==0:
        #     #     pbar.set_description("loss:{}".format(loss))
        #     epoch_loss+=loss.item()
        #     print("Epoch_loss:{}".format(epoch_loss/len(dataloader.dataset)))
        # train_acc = evaluate(net, dataloader)
        # train_list.append(train_acc)
        # print('train_acc', train_acc)

if __name__=="__main__":
    train()





Epoch [0/10]:  20%|█▉        | 17/87 [00:18<01:17,  1.10s/it, acc=0.0938, loss=1.81]


KeyboardInterrupt: 

In [56]:
from tqdm import tqdm,trange,tnrange,tqdm_notebook
from time import sleep

# 普通进度条
for i in trange(60):
	#TODO:
	sleep(.1)

# 专为notebook设计的进度条
for i in tqdm_notebook(range(600)):
	#TODO:
	sleep(.1)





























































































































100%|██████████| 60/60 [00:06<00:00,  9.16it/s]
C:\Users\wang\AppData\Local\Temp\ipykernel_1540\610718300.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(600)):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html